### 전력 수요량 예측 경진대회 코드리뷰 - 2위
- https://dacon.io/competitions/official/196878/codeshare/416?page=1&dtype=recent&ptype=pub

### 배경
- 전력수요예측 시뮬레이션을 통한 효율적인 빅데이터 분석기법 발굴
- 전력 융합신서비스 발굴 및 비즈니스 모델 개발 활용
- 창의적인 아이디어와 빅데이터 분석기술을 토대로 에너지 신서비스 개발 촉진

### 문제
- 본 대회에서는, 기존 전력 사용 기록과 기상 데이터 등 공공 데이터를 이용하여, 각 가정 및 회사의 시간별, 일별, 월별 전력 사용량을 예측합니다. **2018년 7월1일부터 2018년 - 11월 30일까지의 에너지 사용량을 예측**합니다. 보다 정확히는 다음을 예측합니다.
- 2018년 **7월 1일 00시부터 24시까지**, 24시간, ‘시간당 전력사용량’ (24개) 
- 2018년 **7월 1일부터 7월10일까지, 10일간**, ‘일간 전력사용량’ (10개)
- 2018년 **7월 부터 11월까지, 5개월간**, ‘월간 전력사용량’ (5개)
- 즉 각 세대(또는 상가)당 39개(24개,10개,5개)의 값을 예측해야 합니다.

### 데이터
- 본 대회에서는 2016년 7월 20일부터 2018년 6월 30일까지의 국내 특정 지역의 아파트들과 상가의 전력에너지 사용량이 주어집니다. 자세한 사항은 첨부파일로 포함합니다.
- * 주의: 제공되는 데이터에는 결측치나 이상치(NA, 0인 값)가 포함되어 있습니다. 대회참가자들은 이러한 결측치를 고려하여, 결측치 처리 등 예측 기법을 적용하여야 합니다.
- 참고로, NA가 발생한 경우, 직전 시간의 전력사용량 값이 상당히 큰 경향이 있습니다. 이는 미터링 데이터 수집 시스템의 특징으로 보입니다. 그러나 반드시 그런 것은 아닙니다.

#### ① train.csv
- 국내(인천지역) 특정 지역의 모 아파트 및 모 상가의 전력사용량. (1300호)
- 2016년 7월 26일 11시 ~ 2018년 6월 30일 24시까지 시간 당 전력사용량

#### ② test.csv
- 국내(인천지역) 특정 지역의 모 아파트 및 모 상가의 전력사용량. (200호)
- 2017년 7월 1일 00시 ~ 2018년 6월 30일 24시까지 시간 당 전력사용량
* train의 세대와 다른 세대

#### ③ submission.csv
- test셋에 제시된 세대(상가)에 대한 예측값들을 제출하는 포맷
- 본 대회에서는 2016년 7월 20일부터 2018년 6월 30일까지의 국내 특정 지역의 아파트들과 상가의 전력에너지 사용량이 주어집니다. 자세한 사항은 첨부파일로 포함

In [27]:
import pandas as pd             #데이터 전처리
import numpy as np              #데이터 전처리
import os
from sklearn.ensemble import RandomForestRegressor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#%% ============================== < Main > =======================================
#%% Section #1: Loading data...    
#------------------------------------------------------------------------------
# [Output]
# <test> : test.csv
# <submission> : Data frame for submission.
#------------------------------------------------------------------------------ 
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/submission.csv')

# 중간중간 결과 출력해주는 함수
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 원래 object형 -> Time 형으로 변환해줌
test['Time'] = pd.to_datetime(test.Time)
test = test.set_index('Time')

print('Section [1]: Loading data............... Completed')

Section [1]: Loading data............... Completed


In [28]:
print(test.shape)
test.head()

(8760, 200)


,NX1301,NX1302,NX1303,NX1304,NX1305,NX1306,NX1307,NX1308,NX1309,NX1310,...,NX1491,NX1492,NX1493,NX1494,NX1495,NX1496,NX1497,NX1498,NX1499,NX1500
Time,,,,,,,,,,,,,,,,,,,,,
2017-07-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.275,0.021,NaN
2017-07-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.222,0.021,NaN
2017-07-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.237,0.021,NaN
2017-07-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.229,0.020,NaN
2017-07-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.202,0.021,NaN


In [29]:
print(submission.shape)
submission.head()

(200, 40)


,meter_id,X2018_7_1_1h,X2018_7_1_2h,X2018_7_1_3h,X2018_7_1_4h,X2018_7_1_5h,X2018_7_1_6h,X2018_7_1_7h,X2018_7_1_8h,X2018_7_1_9h,...,X2018_7_6_d,X2018_7_7_d,X2018_7_8_d,X2018_7_9_d,X2018_7_10_d,X2018_7_m,X2018_8_m,X2018_9_m,X2018_10_m,X2018_11_m
0,NX1301,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NX1302,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NX1303,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NX1304,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NX1305,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
#============================ <Coding 구성> =======================================
# [Local Function #1]: SAMPE calculation module
# [Local Function #2]: AR_data_set calculation module = 시계열 데이터 추출 모듈
# [Local Function #3]: AR_day_set calculation module = 시계열 day set 추출 모듈
# [Local Function #4]: linear_prediction calculation module = 선형 계산 모듈
# [Local Function #5]: Random forest module = 랜덤 포레스트 모듈
# [Local Function #6]: DNN module = DNN 모듈

# [Main function]
# [Section #1]: Data loading section = 데이터 로딩
# [Section #2]: Data generation for training set = train 데이터 셋 생성
# [Section #3]: Anormaly detection using AR model  = 이상치 탐지
# [Section #4]: data prediction for hour profile = 시간단위의 데이터 예측
# [Section #5]: data prediction for day profile = 일단위의 데이터 예측
# [Section #6]: data prediction for month profile = 월단위의 데이터 예측
#=============================================================================

In [31]:
# %% Local function #1 - <SAMPE calculation module>
#------------------------------------------------------------------------------
# [Input]
# <A> : Real data.
# <F> : Forecasting data.
#------------------------------------------------------------------------------ 
# [Output] # 평가지표 SMAPE로 값 구하기
def smape(A, F): # A는 Real Data, F는 Forecaseting Data
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [32]:
# [Input]
# < Data > : test.csv.
# < place_id > : power meter ID.
# < prev_type > : 예측 전 날의 데이터 타입(<1> : Workday <2> : Weekend)
# < Curr_type >: 예측 날의 데이터 타입(<1> : Workday <2> : Weekend)  
#------------------------------------------------------------------------------ 
# [Output]
# < TrainAR >: 예측 전날의 data set [day x time]
# < TestAR >: 예측 날의 data set [day x time]
#------------------------------------------------------------------------------ 
# %% Local function <AR_data_set calculation module>
# 시간 데이터 예측을 위한 데이터 셋 추출 함수.
# test.csv 파일 내에 전력 데이터를 요일 타입을 고려하여 분류함.
# 전날 데이터를 학습하여 다음날을 예측하는 방식을 사용하며, 요일 타입은 2가지로 
# 분류함. 월~금은 Workday, 토~일은 weekend    
#------------------------------------------------------------------------------

# test.csv, meter ID, 예측 전날의 데이터 타입, 예측 날의 데이터 타입 
def AR_data_set(Data, place_id, prev_type, Curr_type):
    # Mon: 0 ~ Sun:6
    TrainAR = []; TestAR = []
    len_bad = 20  # 하루 내 NaN의 개수 기준, (24-len_bad)보다 많으면 그 날은 제거 
    Power = Data[place_id].iloc  # test.csv에서 특정 id의 전력 데이터 - NX1301, NX1302, NX1303 컬럼들의 행들 이런식으로..
    Date = Data[place_id].index  # test.csv에서 특정 id의 날짜 데이터 - 뽑은 컬럼에 대한 인덱스 (Date Time 으로 변환했으니 Date)
        
    # 예측전날, 예측날들의 데이터들에 대한 다 일단 [0]으로
    prev_aloc = [0]*24;  curr_aloc = [0]*24  # pre-allocation
    
    # 모든 행 점검 - bad data  - 7104 행부터는 제대로 나옴.
    for ii in range(24,len(Date)): # ii 는 24 ~ 전체 행의 갯수
        # Date - datetime 형태의 index => 시간이 자정이거나, 48행이후부터거나, 
        # 과거 1시꺼 데이터 * 24가 prev_aloc 다 합친 것과 같지 않을떄?
        # 현재 1시꺼 데이터 * 24가 curr_aloc 다 합친것과 같지 않을 때?
        
        # 일주일마다 이 if 가 출력됨.
        if (Date[ii].hour == 0) & (ii > 48) & (np.sum(curr_aloc) != 24 * curr_aloc[1]) & (np.sum(prev_aloc) != 24 * prev_aloc[1]): 
            prev_idx = 0;  curr_idx = 0      # bad data idx
            # print(ii, '. prev_aloc, curr_aloc', prev_aloc, curr_aloc) # ii가 168씩 증가한다? => 1주일마다
            
################## 예측 전날에 대한 보간법 - 하루씩 봐야됨
            for kk in range(0,24): # kk 는 하루 24시간
                if prev_aloc[kk]>-20: # check the bad data.
                    prev_idx = prev_idx+1     
                    #print('bad_data', Data[ii])
                    
                # bad data가 아닐 때
                else: # interpolate the bad data.
                    # bad data일 경우, 앞뒤로 20개의 포인트를 가져와서 
                    # interpolation 진행. - 보간법
                    temp = np.zeros([1,41]) 
                    for qq in range(0,41): # Power NX1301 의 행 전체
                        temp[0,qq] = Power[(ii-24)-(24-kk)-20+qq]
                    
                    # df 로 변환
                    temp_temp = pd.DataFrame(data = temp)

                    # data frame 형으로 바꿔서 보간법만 진행하고 다시 값들만 집어넣음.
                    temp = temp_temp.interpolate('spline',order =1) # 보간법 진행
                    temp = temp.values # 값들만
                    prev_aloc[kk] = temp[0,20]
                    
######################### 예측날에 대한 보간법
            for kk in range(0,24): # 하루 24시간?
                if curr_aloc[kk]>-20:       # check the bad data.
                    curr_idx =curr_idx+1
                else:
                    # bad data일 경우, 앞뒤로 20개의 포인트를 가져와서 
                    # interpolation 진행.
                    temp = np.zeros([1,41])
                    for qq in range(0,41):
                        temp[0, qq] = Power[(ii)-(24-kk)-20+qq] # ii 는 30, kk는 12, qq는 -25 ~ 15
                        # temp[0, -25] ~ temp[0,15]
                    
                    # df 로 변환
                    temp_temp = pd.DataFrame(data = temp)
                    
                    # data frame 형으로 바꿔서 보간법만 진행하고 다시 값들만 집어넣음.
                    temp = temp_temp.interpolate('spline',order =1)
                    temp = temp.values
                    curr_aloc[kk] = temp[0,20]

######################### 
            # bad data가 특정 개수 이상이면, data set에 추가하지 않는다.  
            # TrainAR, TestAR 다시 구성한 것.
            if (prev_idx>len_bad)&(curr_idx>len_bad):
                TrainAR.append(prev_aloc)
                TestAR.append(curr_aloc)
                        
        # 0시에 하루 데이터 초기화.                     
        if Date[ii].hour == 0:
            prev_aloc = [0]*24
            curr_aloc = [0]*24
        
        # 요일 데이터 확인.
        prev_day = Date[ii-24].weekday() # 하루전날의 요일
        curr_day = Date[ii].weekday() # 현재날의 요일
        
        # 요일 데이터 타입 분류
        # Workday(1) = day type<5(월~금) - 평일
        # Workday(2) = day type>=5(토~일) - 주말
        
########### 하루전날에 대한 전력데이터, 현재에 대한 전력데이터 prev_aloc, curr_aloc로 구함.
        # 전날이 주말아니고 요일(월-금)이고, 현재날짜가 주말이고, 요일(토-일) 일 때 ==> 평일-주말
        if ((prev_type ==1) & (prev_day<5)) & ((Curr_type ==2) & (curr_day>4)): # <1> : Workday <2> : Weekend        
            prev_aloc[Date[ii-24].hour] = Power[ii-24] # 하루전날에 대한 전력데이터
            curr_aloc[Date[ii].hour] = Power[ii] # 현재에 대한 전력데이터
        
        # 전날이 주말아니고 요일(월-금)이고, 현재날짜가 평일이고, 요일(월-금) 일 때 == > 평일-평일
        if ((prev_type ==1)&(prev_day<5))&((Curr_type ==1)&(curr_day<5)): # <1> : Workday <2> : Weekend
            prev_aloc[Date[ii-24].hour] = Power[ii-24]
            curr_aloc[Date[ii].hour] = Power[ii]
            
        # 전날이 주말이고 요일(토-일)이고, 현재날짜가 주말이고, 요일(토-일) 일 때 ==> 주말-주말
        if ((prev_type ==2)&(prev_day>4))&((Curr_type ==2)&(curr_day>4)): # <1> : Workday <2> : Weekend
            prev_aloc[Date[ii-24].hour] = Power[ii-24]
            curr_aloc[Date[ii].hour] = Power[ii]
        
        # 전날이 주말이고 요일(토-일이고, 현재날짜가) 평일이고, 요일 (월-금)일 때 ==> 주말-평일
        if ((prev_type ==2)&(prev_day>4))&((Curr_type ==1)&(curr_day<5)):           
            prev_aloc[Date[ii-24].hour] = Power[ii-24]
            curr_aloc[Date[ii].hour] = Power[ii]

    TrainAR = np.array(TrainAR)
    TestAR = np.array(TestAR)
    return TrainAR, TestAR

In [33]:
# %% Local function < AR_day_set calculation module >
# 하루 사용 데이터 예측을 위한 데이터 셋 추출 함수.
# test.csv 파일 내에 전력 데이터를 요일 타입을 고려하여 분류함.

# <요일 타입>: 월 ~ 일
# Similar day approach method만을 활용할 예정이기 때문에 최근 데이터 6주만을 
# 정리함.    
#------------------------------------------------------------------------------
# [Input]
# <Data> : test.csv.
# <place_id>: power meter ID.
#------------------------------------------------------------------------------ 
# [Output]
# <temp_day>: 과거 하루 전력 사용량  [week number(최근 순) x day type]
#------------------------------------------------------------------------------     
def AR_day_set(data, place_id):
    Power = data[place_id].values       #전력 데이터 
    Date = data[place_id].index         #요일 데이터 
    
    temp_day = np.zeros([6, 7])         # 6행 7열 짜리 pre-allocation for output dataset
    mon_idx = np.zeros([1, 7])          # 1행 7열 짜리 몇 번째 week인지 확인하는 idx
    
    # 6주로 보는건 => 월단위가 길어봤자 6주
    # 행에서 500을 왜 뻄?  
    for ii in range(0,len(Power)-500):
        idx = len(Power) - ii -1 # index 
        day_idx = Date[idx].weekday()       # data의 요일정보
        time_idx = Date[idx].hour           # data의 시간정보 
        
        if mon_idx[0, day_idx] < 6:         # 6번째 week 이상이면 추가 X
            
            # nan 값 처리. 1주전, 2주전, 3주전 것을 사용.
            if np.isnan(Power[idx]):        # bad data restortion
                res_data = np.zeros([1, 9]) # 1행 9열
                
                # 1주전, 2주전, 3주전의 같은 요일, 시간 데이터를 저장 후 mean
                res_data[0,0] = Power[idx-24*7-1]
                res_data[0,1] = Power[idx-24*7]
                res_data[0,2] = Power[idx-24*7+1]
                
                res_data[0,3] = Power[idx-48*7-1]
                res_data[0,4] = Power[idx-48*7]
                res_data[0,5] = Power[idx-48*7+1]
                
                res_data[0,6] = Power[idx-1]
                res_data[0,7] = Power[idx-3*24*7]
                res_data[0,8] = Power[idx+1]
                
                # 하루 사용량 저장을 위한 시간 데이터 합 - nanmean = nan 값 무시하고 mean 집어넣음.
                # temp_day의 해당 주차의 
                temp_day[int(round(mon_idx[0,day_idx])), day_idx] = temp_day[int(round(mon_idx[0,day_idx])), day_idx]+ np.nanmean(res_data)
    
            # nan 값 보정 처리 필요 없을 때
            else:
                # 하루 사용량 저장을 위한 시간 데이터 합 - 전력값 합.
                temp_day[int(round(mon_idx[0,day_idx])), day_idx] = temp_day[int(round(mon_idx[0,day_idx])), day_idx] + Power[idx]
    
            # 
            if time_idx == 0:
                # 요일이 지나면, week확인 idx +1
                mon_idx[0,day_idx] = mon_idx[0,day_idx] + 1
    return temp_day

In [34]:
#########################################################################################################
#########################################################################################################
#########################################################################################################
#########################################################################################################
#########################################################################################################
#########################################################################################################
#########################################################################################################
#########################################################################################################
#########################################################################################################
#########################################################################################################
#########################################################################################################

In [35]:
#%% Local function <linear_prediction calculation module>
### 자기회귀모델인듯? !
# 선형 예측 방식 구현. (Autoregressive model) - 자기회귀모델
# Y(예측 날) = A(coefficient) * X(예측전날)
# X-1(역행렬)*Y = A
# A를 추출하여, 예측에 활용함.     
#------------------------------------------------------------------------------
# [Input]
# < trainAR >: test.csv의 예측 전날 데이터 셋 
# < testAR >: test.csv의 예측 날 데이터 셋 
# < flen >: filter length(예측전날의 몇개의 데이터를 가져다 쓸지 결정)
# < test_data > : 실제로 예측 하고 싶은 전날의 데이터(TEST 데이터)    
#------------------------------------------------------------------------------ 
# [Output]
# < avr_smape > : Training set으로 테스트 했을 때, smape
# < fcst > : test_data의 예측 결과 
# < pred > : Traing set으로 예측했던 예측 결과    
#------------------------------------------------------------------------------   
# 예측전날 데이터셋 / 예측 날 데이터셋 / 예측전날에 몇개의 데이터를 가져다 쓸지 / 실제로 예측하고 싶은 TEST 데이터

# 자기회귀모델을 돌려서 
# 예측 날 전기 사용량 (Y) = 계수 * 예측 전날 전기 사용량
def linear_prediction(trainAR, testAR, flen, test_data):
    len_tr = len(trainAR[0,:])   # 시간 포인트 수 
    day_t = len(trainAR)
    pred = np.empty((len(trainAR),len_tr))
    fcst = np.empty((len(trainAR),len_tr))
    
    for j in range(0, day_t):
        if day_t>1:
            x_ar=np.delete(trainAR[:,len_tr-flen:len_tr], (j), axis=0)
            y=np.delete(testAR, (j), axis=0)
        else:
            x_ar = trainAR[:,len_tr-flen:len_tr]
            y = testAR
            
        pi_x_ar = np.linalg.pinv(x_ar)
        lpc_c = np.empty((len(x_ar),flen))
        lpc_c=np.matmul(pi_x_ar, y)
        
        test_e = trainAR[j,:]
        test_ex = test_e[len_tr-flen:len_tr]
        pred[j,:]=np.matmul(test_ex, lpc_c)  
        
    x_ar = trainAR[:,len_tr-flen:len_tr]
    y = testAR
    pi_x_ar = np.linalg.pinv(x_ar)
    lpc_c = np.empty((len(x_ar),flen))
    lpc_c=np.matmul(pi_x_ar, y)
    Test_AR = testAR[0:len(testAR),:]
    smape_list=np.zeros((len(pred),1))

    for i in range(0,len(pred)):
        smape_list[i]=smape(pred[i,:], Test_AR[i,:])
        avr_smape = np.mean(smape_list)  
    
    test_e = test_data
    test_ex = test_e[len_tr-flen:len_tr]   
    fcst = np.matmul(test_ex,lpc_c)
    
    return avr_smape, fcst, pred

In [36]:
#%% Local function < Similar day approach module >
# Similar day approaach method 구현.
# 같은 요일 타입의 날의 데이터를 N개를 추출하여 평균을 취하여 사용함.    
#------------------------------------------------------------------------------
# [Input]
# < trainAR >: test.csv의 예측 전날 데이터 셋 
# < testAR > : test.csv의 예측 날 데이터 셋 
# < slen >: 추출한 날의 수 (N)
# < sim_set >: 실제로 예측 하고 싶은 전날의 데이터(TEST 데이터)    
#------------------------------------------------------------------------------ 
# [Output]
# < simil_smape >: Training set으로 테스트 했을 때, smape
# < simil_temp > : test_data의 예측 결과   
#------------------------------------------------------------------------------
def similar_approach(trainAR, testAR, slen, sim_set):
    simil_smape_list = np.zeros([1,len(testAR[:,0])])
    
    for col_ii in range(0,len(testAR[:,0])):    
        simil_mean = []
        simil_temp =np.zeros([1,24])
        simil_idx =np.zeros([1,len(testAR[:,0])])
        
        for sub_col in range(0,len(testAR[:,0])):
            simil_idx[0,sub_col] = smape(trainAR[col_ii,:],trainAR[sub_col,:])
            
        testAR_temp = np.delete(testAR, np.argmin(simil_idx), axis=0)
        simil_idx = np.delete(simil_idx, np.argmin(simil_idx), axis=1)
        
        for search_len in range(0,slen):
            simil_mean.append(testAR_temp[np.argmin(simil_idx), :])
            testAR_temp = np.delete(testAR, np.argmin(simil_idx), axis=0)
            simil_idx = np.delete(simil_idx, np.argmin(simil_idx), axis=1)
              
        for row_ii in range(0, 24):           
            simil_temp[0, row_ii] = np.median(testAR_temp[:,row_ii])

        simil_smape_list[0,col_ii] = smape(testAR[col_ii,:], simil_temp)
        simil_smape  = np.mean(simil_smape_list)
    
    simil_mean = []
    simil_temp =np.zeros([1,24])
    simil_idx =np.zeros([1,len(testAR[:,0])])
    testAR_temp = testAR
    
    for sub_col in range(0,len(testAR[:,0])):
        simil_idx[0,sub_col] = smape(sim_set,trainAR[sub_col,:])

    for search_len in range(0,slen):
        simil_mean.append(testAR_temp[np.argmin(simil_idx),:])
        testAR_temp = np.delete(testAR, np.argmin(simil_idx), axis=0)
        simil_idx = np.delete(simil_idx, np.argmin(simil_idx), axis=1)
   
    for row_ii in range(0, 24):           
        simil_temp[0, row_ii] = np.median(testAR_temp[:,row_ii])
    
    return simil_smape, simil_temp

In [37]:
#%% Local function < Random forest module > 
# Random forest를 이용한 regression 기반의 forecasting algorithm
# 전날의 24시간 프로파일을 이용해 다음날 24시간의 프로파일을 예측하는 시스템.   
#------------------------------------------------------------------------------
# [Input]
# < trainAR > : test.csv의 예측 전날 데이터 셋 
# < testAR > : test.csv의 예측 날 데이터 셋 
# < x_24hrs > : 실제로 예측 하고 싶은 전날의 데이터(TEST 데이터)  
#------------------------------------------------------------------------------ 
# [Output]
# < ypr > : x_24hrs를 이용한 예측 결과  
# < avr_smape > : validation set으로 확인한, smape      
#------------------------------------------------------------------------------
def machine_learn_gen(trainAR, testAR, x_24hrs):
    Dnum = trainAR.shape[0]; lnum = trainAR.shape[1]
    smape_list = np.zeros([Dnum,1])
    
    for ii in range(0, Dnum): # cross validation
        trainAR_temp = np.delete(trainAR, ii, axis=0)
        testAR_temp  = np.delete(testAR, ii, axis=0)
        
        # mae 기반의 loss를 이용한 randomforest model 생성
        regr = RandomForestRegressor(max_depth=2, random_state=0,n_estimators=100, criterion='mae')
        regr.fit(trainAR_temp, testAR_temp)

        x_temp = np.zeros([1,lnum])
        for kk in range(0,lnum):
            x_temp[0,kk] = trainAR[ii, kk]
            
        ypr = regr.predict(x_temp)

        yre = np.zeros([1,lnum])
        for kk in range(0,lnum):
            yre[0,kk] = testAR[ii, kk]
        
        smape_list[ii] = smape(np.transpose(ypr),np.transpose(yre))
        
    regr = RandomForestRegressor(max_depth=2, random_state=0,n_estimators=100, criterion='mae')
    regr.fit(trainAR, testAR)
        
    x_24hrs = np.reshape(x_24hrs,(-1,lnum))
    
    avr_smape = np.mean(smape_list)
    ypr=regr.predict(x_24hrs)
    
    return ypr,  avr_smape, smape_list

In [38]:
# %% Local function < DNN module >
# DNN를 이용한 regression 기반의 forecasting algorithm
# 전날의 24시간 프로파일을 이용해 
# 다음날 24시간의 프로파일을 예측할 수 있는 DNN 모델 생성.    
#------------------------------------------------------------------------------
# [Input]
# < trainAR >: test.csv의 예측 전날 데이터 셋 
# < testAR > : test.csv의 예측 날 데이터 셋 
# < EPOCHS > : DNN의 학습을 위한 epoch size   
#------------------------------------------------------------------------------
# [Output]
# < model > : 학습된 모델 출력  
# < avr_smape > : test set으로 테스트 했을 때의 평균, smape  
# < smape_list >: test set으로 테스트 했을 때의 각각, smape      
#------------------------------------------------------------------------------     
def non_linear_model_gen(trainAR, testAR, EPOCHS):
    numData=np.size(trainAR,0)
    numTr=int(numData*0.8)
    Xtr=trainAR[0:numTr-1,:]
    Ytr=testAR[0:numTr-1,:]
    
    Xte=trainAR[numTr:numData,:]
    Yte=testAR[numTr:numData,:]
    
    num_tr = np.size(trainAR,1)
    num_te = np.size(testAR,1)
    
    def build_model():        
        model = keras.Sequential([
            layers.Dense(128, activation='relu', input_shape=(num_tr,)),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(num_te)
        ])

        optimizer = tf.keras.optimizers.Adam(0.001)
        
        model.compile(loss='mae',
                        optimizer=optimizer,
                        metrics=['mae', 'mse'])
        return model

    model = build_model()
    # model.summary()
    
    #example_batch = Xtr[:10]
    #example_result = model.predict(example_batch)
    #example_result
    
    class PrintDot(keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs):
            if epoch % 100 == 0: print('')

    history = model.fit(
      Xtr, Ytr,
      epochs=EPOCHS, verbose=0,
      callbacks=[PrintDot()])
   
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    # hist.tail()
    
    Ypr = model.predict(Xte)
    
    smape_list=np.zeros((len(Ypr),1))
    
    for i in range(0,len(Ypr)):
        smape_list[i]=smape(Ypr[i,:], Yte[i,:])
    avr_smape=np.mean(smape_list)
    
    return model, avr_smape

In [39]:
#########################################################################################################
#########################################################################################################
#########################################################################################################
#########################################################################################################
#########################################################################################################
#########################################################################################################
#########################################################################################################
#########################################################################################################
#########################################################################################################
#########################################################################################################
#########################################################################################################

In [ ]:
#%% ============================== < Main > =======================================
#%% Section #2: Data generation for training set  
#------------------------------------------------------------------------------
# [Input]
# < test > : Test set
# < submission > : Data frame for submission.
# < prev_type > : Day type of the previous day (workday = 1, weekend =2). # 이전날의 요일 type
# < curr_type > : Day type of the current day (workday = 1, weekend =2). # 현재날의 요일 type
#------------------------------------------------------------------------------ 
# [Output]
# < testAR > : Test set
# < trainAR > : Training set
# < subm_24hrs > test data for 24hrs prediction
#------------------------------------------------------------------------------ 
agg = {}; comp_smape =[]; key_idx = 0
for key in test.columns:
    key_idx = key_idx + 1
    print('key, key_idx', [key, key_idx])
    prev_type = 2   # 전날 요일 타입
    curr_type = 2   # 예측날 요일 타입
    
    # shape -> trainAR(10,24) / testAR(10,24)
    trainAR, testAR = AR_data_set(test, key, prev_type, curr_type) 
    
    print('Section [2]: Data generation for training set...............')
    
    ### [시간 예측을 위한 마지막 24pnt 추출]
    # NaN 값처리를 위해서 마지막 40pnts 추출 한 후에 
    temp_test = test[key].iloc[8759-40:] # 끝에서 40개를 왜 보간법 다시?
    temp_test = temp_test.interpolate(method='spline', order=2) # 보간법 - nan값 있을떄만 다시 채워주는듯? 

    # interpolation하고 나서 24pnts 재추출 
    temp_test = np.array(temp_test.values)
    temp_test = temp_test[len(temp_test)-24:len(temp_test)+1] # 40개 중에 24개 재추출
    subm_24hrs = temp_test
    del temp_test
    
# %% Section #3: Anormaly detection using AR model  = 이상치 탐지
#------------------------------------------------------------------------------
# [Input] : # < testAR > : Test set   # < trainAR >: Training set
#------------------------------------------------------------------------------
# [Output] : # < testAR > : Test set   # < trainAR >: Training set
#------------------------------------------------------------------------------   
    fchk = 1        # filter length
    temp_idx = []; smape_lin = []
    
################## 이상치 탐지중 - Nan 값 제거
    # 한 행씩 linear prediction을 테스트해보고 NaN이 발견된다면, 그 행을 제거.
    for chk_bad in range(0, len(trainAR[:,0])):
        prev_smape = 200 # SMAPE 기준값 
        nan_chk = 0      # NaN chk idx
        trainAR_temp = np.zeros([1,24])     # pre-allocation
        testAR_temp = np.zeros([1,24])      # pre-allocation

        # 한 행씩 테스트를 하기 위한 변수 설정
        for ii in range(0,24):
            trainAR_temp[0,ii] = trainAR[chk_bad,ii] # 한행씩 체크중
            testAR_temp[0,ii] = testAR[chk_bad,ii] # 한행씩 체크중
        
        # linear prediction test - lin_sampe = () / fcst_temp = (24, ) / pred_hyb = (1, 24)
        # samp 값, 미래값, 예측값
        lin_sampe, fcst_temp, pred_hyb = linear_prediction(trainAR_temp, testAR_temp, fchk, subm_24hrs)
            
        if np.isnan(lin_sampe):     # SMAPE가 NaN 경우, 그 행을 제거
            nan_chk = 1
        if np.isnan(np.sum(trainAR_temp)): # chk_bad의 행이 NaN을 포함할 경우 제거
            nan_chk = 1         
        if np.isnan(np.sum(testAR_temp)): # chk_bad의 행이 NaN을 포함할 경우 제거
            nan_chk = 1
        if nan_chk == 1: #NaN 값이 있는 행 넘버를 append
            temp_idx.append(chk_bad)

    # NaN 값이 나타난 data set은 제거 
    trainAR = np.delete(trainAR, temp_idx, axis=0)
    testAR = np.delete(testAR, temp_idx, axis=0)                 
    # print('NaN 값이 나타난 행 (제거)', temp_idx) - > 다양한 항이 제거됨

########################## filter length 최적화
    del_smape = np.zeros([1,len(trainAR[:,1])])
    prev_smape = 200
    fchk = 0
    
    # filter length 최적화 
    for chk in range(3,24):
        # filter length을 바꿔가며 Smape가 최소가 되는 값을 찾아감.
        lin_sampe, fcst_temp, pred_hyb = linear_prediction(trainAR, testAR, chk, subm_24hrs)
        if prev_smape>lin_sampe:
            fchk = chk
            prev_smape = lin_sampe    
   
    # 필요없는 데이터 제거
    # 한 줄(하루)씩 제거해가면서 SMAPE 결과를 분석.
    for chk_lin in range(0,len(trainAR[:,1])):
        trainAR_temp = np.delete(trainAR, chk_lin, axis=0)
        testAR_temp = np.delete(testAR, chk_lin, axis=0)
        lin_sampe, fcst_temp, pred_hyb = linear_prediction(trainAR_temp, testAR_temp, fchk, subm_24hrs)          
         
        del_smape[0,chk_lin] = lin_sampe
    
    # SMAPE에 악영향을 주는 행을 제거      
    trainAR = np.delete(trainAR, np.argmin(del_smape), axis=0)
    testAR = np.delete(testAR, np.argmin(del_smape), axis=0)
    del_smape = np.delete(del_smape, np.argmin(del_smape), axis =1)
    print('Section [3]: mitigating bad data...............')
    del nan_chk, lin_sampe, fcst_temp, pred_hyb, prev_smape, temp_idx
    
#%% Section #4: Prediction test
#------------------------------------------------------------------------------
# [Output]
# <fcst> : Predicted hour profile result 
#------------------------------------------------------------------------------ 
    # DNN model 
    EPOCHS = 1 # 80
    Non_NNmodel, non_smape = non_linear_model_gen(trainAR, testAR, EPOCHS)
    
    # random forest model
    mac_fcst, Mac_smape, smape_listss = machine_learn_gen(trainAR, testAR,subm_24hrs)
    
    # linear model
    lin_sampe, fcst_temp, pred_hyb = linear_prediction(trainAR, testAR, fchk, subm_24hrs)
    
    # Similar day approach model
    temp_24hrs = np.zeros([1,24])    # np.array type으로 변경. 
    for qq in range(0,24):
        temp_24hrs[0,qq] = subm_24hrs[qq]
    
    # Similar day approach model 최적화 (몇 개의 날(N)을 가져오는 게 좋은 지 평가.)    
    prev_smape = 200
    fsim = 0  # N개의 날 
    for sim_len in range(2, 5):    
        sim_smape,  fcst_sim = similar_approach(trainAR, testAR, sim_len, temp_24hrs)
        if prev_smape>sim_smape:
            fsim = sim_len
            prev_smape = sim_smape
            
    # Similar day approach model       
    sim_smape,  fcst_sim = similar_approach(trainAR, testAR, fsim, temp_24hrs)
    # ---------------------------------------------------------------------------------------    
    
    minor_idx = 0 # Autoregression model에서 minor value가 나타나면, 
    # 모델을 Autoregression model에서 similar day appreach로 변경 진행.
    
    # SMAPE가 linear model이 가장 작으면, 해당 결과 사용
    if (lin_sampe<non_smape)&(lin_sampe<Mac_smape)&(lin_sampe<sim_smape):    
        fcst = np.zeros([1,24])  
        for qq in range(0,24):
            fcst[0,qq] = fcst_temp[qq]
            
            if fcst_temp[qq]<0:
                minor_idx = minor_idx+1
                
    # SMAPE가 DNN model이 가장 작으면, 해당 결과 사용
    if (non_smape<lin_sampe)&(non_smape<Mac_smape)&(non_smape<sim_smape):
        temp_24hrs = np.zeros([1,24])
        for qq in range(0,24):
            temp_24hrs[0,qq] = subm_24hrs[qq]
            
        fcst = Non_NNmodel.predict(temp_24hrs)
    
    # SMAPE가 random forest model이 가장 작으면, 해당 결과 사용
    if (Mac_smape<non_smape)&(Mac_smape<lin_sampe)&(Mac_smape<sim_smape):
        fcst = mac_fcst
    
    # SMAPE가 Similar day approach model이 가장 작으면, 해당 결과 사용        
    if (sim_smape<non_smape)&(sim_smape<lin_sampe)&(sim_smape<Mac_smape):
        fcst = fcst_sim
        
    if (minor_idx>0):
        fcst = fcst_sim
    
    # 각 SMAPE 결과 값을 정    
    comp_smape.append([non_smape, lin_sampe,Mac_smape, sim_smape])

    
    a = pd.DataFrame() # a라는 데이터프레임에 예측값을 정리합니다.
    
    print('Section [4]: Hour prediction model...............')    
    for i in range(24):
        a['X2018_7_1_'+str(i+1)+'h']=[fcst[0][i]] # column명을 submission 형태에 맞게 지정합니다.
        

#%% Section #5: Day prediction
#------------------------------------------------------------------------------
# [Output]
# < fcst_d >: Predicted day profile (result)
#------------------------------------------------------------------------------ 
    fcst_d = np.zeros([1,10])       # pre-allocation of the result data
    trainAR_Day = AR_day_set(test, key)  #데이터를 불러옵니다.
    
    # Similar day aprroach
    day_idx = np.zeros([1, 10])
    for ii in range(0, 10):
        mod_idx = -1
        temp_idx =  (ii+mod_idx) % 7     #예측하는 날에 맞는 요일 idx를 정리.
        day_idx[0, ii]  = temp_idx
        
    for ii in range(0,10):
        flen = np.random.randint(3)+2    # 2~5개까지 랜덤하게 과거 데이터를 불러옵니다.
        temp_day = np.zeros([1, flen])
        for jj in range(0, flen):  
            temp_day[0,jj] =  trainAR_Day[jj, int(round(day_idx[0, ii]))]
        
        # 불러온 데이터를 평균을하여 예측함.
        fcst_d[0,ii] = np.mean(temp_day)
    print('Section [5]: Day prediction model...............')
    
    for i in range(10):
        a['X2018_7_'+str(i+1)+'_d']=[fcst_d[0][i]] # column명을 submission 형태에 맞게 지정합니다.
    
    del mod_idx, temp_idx
#%% Section #6: Month prediction
#------------------------------------------------------------------------------
# [Output]
# < pred_(N)m >: N번째 달의 전력 사용량 예측 결과
#------------------------------------------------------------------------------ 
    mon_test = np.zeros([1,300])
    
    # Similar day aprroach 
    day_idx = np.zeros([1, 300])
    for ii in range(0, 300):
        mod_idx = -1
        temp_idx = (ii+mod_idx) % 7  # 요일 idx 생성(월~일: 0~6)
        day_idx[0, ii]  = temp_idx
    
    # 휴일의 경우, 일요일과 같은 데이터로 가정함.
    day_idx[0,31+15-1] = 6 # 광복절
    day_idx[0,31+31+24-1] = 6 # 추석
    day_idx[0,31+31+25-1] = 6 # 추석
    day_idx[0,31+31+26-1] = 6 # 대체휴일
    day_idx[0,31+31+30+3-1] = 6 # 개천절
    day_idx[0,31+31+30+9-1] = 6 # 한글날
    day_idx[0,31+31+30+31+30+25-1] = 6 # 성탄절
    
    for ii in range(0,300):
        flen = np.random.randint(3)+1  # Similar day approach를 위한 1~4개의 데이터 추출
        temp_day = np.zeros([1, flen])
        for jj in range(0, flen):  
             temp_day[0,jj] =  trainAR_Day[jj, int(round(day_idx[0, ii]))]
            
        mon_test[0,ii] = np.mean(temp_day)

    # 결과 합         
    pred_7m = np.sum(mon_test[0,0:31])
    pred_8m = np.sum(mon_test[0,31:62])
    pred_9m = np.sum(mon_test[0,62:92])
    pred_10m = np.sum(mon_test[0,92:123])
    pred_11m = np.sum(mon_test[0,123:153])
    
    a['X2018_7_m'] = [pred_7m] # 7월
    a['X2018_8_m'] = [pred_8m] # 8월
    a['X2018_9_m'] = [pred_9m] # 9월
    a['X2018_10_m'] = [pred_10m] # 10월
    a['X2018_11_m'] = [pred_11m] # 11월 
  
    a['meter_id'] = key 
    agg[key] = a[submission.columns.tolist()]  
    
    print('Section [6]: Month prediction model...............')


In [43]:

#%% Section #6: Write the result data...
# makes the result file
# [File name]: dacon_submmision.csv

output1 = pd.concat(agg, ignore_index=False)
output2 = output1.reset_index().drop(['level_0','level_1'], axis=1)
output2['id'] = output2['meter_id'].str.replace('NX','').astype(int)
output2 =  output2.sort_values(by='id', ascending=True).drop(['id'], axis=1).reset_index(drop=True)
output2.to_csv('sub_baseline.csv', index=False)

print('Section [7]: Saving the result files...............')

Section [7]: Saving the result files...............


![모델링1](img/모델링1.png)
![모델링2](img/모델링2.png)
![ar모델](img/ar모델.png)
![similar모델](img/similar모델.png)
![rn_and_dnn모델](img/rf_and_dnn모델.png)